In [8]:
import pandas as pd
import numpy as np

# 1. Загрузите выборку из файла titanic.csv с помощью пакета Pandas.
df = pd.read_csv('titanic.csv', sep=',')
# 2. Оставьте в выборке четыре признака: класс пассажира (Pclass), цену билета (Fare), возраст пассажира (Age) и его пол (Sex).
dataset = pd.DataFrame(df,columns  = ['Pclass','Fare','Age','Sex','Survived'])
# 5. В данных есть пропущенные значения — например, для некоторых
# пассажиров неизвестен их возраст. Такие записи при чтении их в
# pandas принимают значение nan. Найдите все объекты, у которых
# есть пропущенные признаки, и удалите их из выборки.
dataset = dataset.dropna()
# 3. Обратите внимание, что признак Sex имеет строковые значения.
dataset['Sex'].replace(['male', 'female'], [1, 0], inplace=True)


#как преобразовать в массив numpy для деревьев
dataset = np.array(dataset)
print(dataset)
dataset = dataset.tolist()
print(dataset)


[[ 3.      7.25   22.      1.      0.    ]
 [ 1.     71.2833 38.      0.      1.    ]
 [ 3.      7.925  26.      0.      1.    ]
 ...
 [ 1.     30.     19.      0.      1.    ]
 [ 1.     30.     26.      1.      1.    ]
 [ 3.      7.75   32.      1.      0.    ]]
[[3.0, 7.25, 22.0, 1.0, 0.0], [1.0, 71.2833, 38.0, 0.0, 1.0], [3.0, 7.925, 26.0, 0.0, 1.0], [1.0, 53.1, 35.0, 0.0, 1.0], [3.0, 8.05, 35.0, 1.0, 0.0], [1.0, 51.8625, 54.0, 1.0, 0.0], [3.0, 21.075, 2.0, 1.0, 0.0], [3.0, 11.1333, 27.0, 0.0, 1.0], [2.0, 30.0708, 14.0, 0.0, 1.0], [3.0, 16.7, 4.0, 0.0, 1.0], [1.0, 26.55, 58.0, 0.0, 1.0], [3.0, 8.05, 20.0, 1.0, 0.0], [3.0, 31.275, 39.0, 1.0, 0.0], [3.0, 7.8542, 14.0, 0.0, 0.0], [2.0, 16.0, 55.0, 0.0, 1.0], [3.0, 29.125, 2.0, 1.0, 0.0], [3.0, 18.0, 31.0, 0.0, 0.0], [2.0, 26.0, 35.0, 1.0, 0.0], [2.0, 13.0, 34.0, 1.0, 1.0], [3.0, 8.0292, 15.0, 0.0, 1.0], [1.0, 35.5, 28.0, 1.0, 1.0], [3.0, 21.075, 8.0, 0.0, 0.0], [3.0, 31.3875, 38.0, 0.0, 1.0], [1.0, 263.0, 19.0, 1.0, 0.0], [1.0, 27.7208

In [9]:
# алгоритм CART (Classification and Regression Tree)
from random import seed
from random import randrange
from csv import reader

# Разделить набор данных на k 
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Рассчитать процент точности
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Оценить алгоритм с использованием разделения перекрестной проверки
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


In [10]:
# Разделить набор данных на основе атрибута и значения атрибута
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] <= value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Рассчитать индекс Джини для набора разделенных данных
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Выбор лучшей точки разделения для набора данных
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups,'count_value_class':[len(g) for g in b_groups]}

# Создать значение терминального узла (листа)
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Создать дочерние разбиения для узла или сделать терминал (лист)
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

# Построить дерево решений
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

# Сделать прогноз с помощью дерева решений
def predict(node, row):
    if row[node['index']] <= node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

# Алгоритм
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    print("-----------------------------------------------------")
    print_tree(tree)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)

# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d <= %.3f]' % ((depth*'   ', (node['index']+1), node['value'])),node["count_value_class"])
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*'   ', node)))

In [11]:
seed(241)

# Оценка построенного дерева
n_folds = 5
max_depth = 7
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Важность: %s' % scores)
print('Точность: %.3f%%' % (sum(scores)/float(len(scores))))

-----------------------------------------------------
[X4 <= 0.000] [209, 359]
   [X1 <= 2.000] [130, 79]
      [X3 <= 2.000] [2, 128]
         [0.0]
         [X2 <= 28.712] [54, 74]
            [X2 <= 27.750] [53, 1]
               [X3 <= 54.000] [51, 2]
                  [X3 <= 36.000] [38, 13]
                     [1.0]
                     [1.0]
                  [0.0]
               [0.0]
            [X2 <= 146.521] [61, 13]
               [X1 <= 1.000] [52, 9]
                  [X1 <= 1.000] [52, 0]
                     [1.0]
                     [1.0]
                  [1.0]
               [X2 <= 151.550] [1, 12]
                  [0.0]
                  [X1 <= 1.000] [12, 0]
                     [1.0]
                     [1.0]
      [X2 <= 20.525] [62, 17]
         [X3 <= 16.000] [16, 46]
            [X3 <= 14.000] [11, 5]
               [X2 <= 10.463] [3, 8]
                  [0.0]
                  [1.0]
               [1.0]
            [X3 <= 36.000] [41, 5]
               

-----------------------------------------------------
[X4 <= 0.000] [204, 364]
   [X1 <= 2.000] [123, 81]
      [X3 <= 2.000] [1, 122]
         [0.0]
         [X2 <= 28.712] [56, 66]
            [X2 <= 27.750] [55, 1]
               [X3 <= 55.000] [53, 2]
                  [X3 <= 36.000] [41, 12]
                     [1.0]
                     [1.0]
                  [0.0]
               [0.0]
            [X2 <= 134.500] [52, 14]
               [X1 <= 1.000] [43, 9]
                  [X1 <= 1.000] [43, 0]
                     [1.0]
                     [1.0]
                  [1.0]
               [X2 <= 151.550] [2, 12]
                  [0.0]
                  [X1 <= 1.000] [12, 0]
                     [1.0]
                     [1.0]
      [X2 <= 19.258] [60, 21]
         [X3 <= 27.000] [46, 14]
            [X3 <= 25.000] [41, 5]
               [X2 <= 15.246] [32, 9]
                  [X2 <= 7.750] [8, 24]
                     [1.0]
                     [0.0]
                  [1.0]
